In [8]:
import math

from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.corpus import stopwords

import os,glob
documents = {}
import requests
import datetime
from bs4 import BeautifulSoup

url = 'https://www.cnn.com/studentnews/article/sitemap-2020-4.html'
currentDate = datetime.datetime.now().strftime("%Y-%m-%d")
limit = 2
keyword = "coronavirus"

print("********************* INPUT **********************")
print("URL: " + url)
print("Date: " + currentDate)
print("No. of Articles: " + str(limit))
print("Keywords: " + keyword)

print("********************* Extracted OUTPUT Content **********************")
code = requests.get(url)
plain = code.text
s = BeautifulSoup(plain, "html.parser")

i=0
for span in s.findAll('span', {'class':'date'}):
    if span.text == currentDate:
        if keyword in span.next_sibling.text:
            if i >= limit:
                break
            i+=1
            print(i)
            href = span.next_sibling.a.get('href')
            article_code = requests.get(href)
            article_plain = article_code.text
            article_soap = BeautifulSoup(article_plain, "html.parser")
            document = ''
            #print(article_soap)
            for span in article_soap.findAll('div', {'class':'zn-body__paragraph'}):
                document += span.text
            documents[document[:15]] = document
            print(document)
            print("**************************************************")

def _create_frequency_matrix(documents):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for document in documents:
        freq_table = {}
        words = word_tokenize(documents[document])
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[document] = freq_table

    return frequency_matrix


def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for document, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_document = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_document

        tf_matrix[document] = tf_table

    return tf_matrix


def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for document, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table


def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for document, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[document] = idf_table

    return idf_matrix


def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (doc1, f_table1), (doc2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[doc1] = tf_idf_table

    return tf_idf_matrix


def _score_sentences(tf_idf_matrix, documents) -> dict:
 
    sentenceValue = {}
    sentences = {}
    total_sentences_in_all_documents = 0
    for document, f_table in tf_idf_matrix.items():
        
        sentences_tokenized = sent_tokenize(documents[document]) # NLTK function        
        for sentence in sentences_tokenized :
            
            total_score_per_sentence = 0
            words = word_tokenize(sentence)
            count_words_in_sentence = len(words)
            for word in words:
                for search_word, score in f_table.items():
                    if search_word == word :
                        total_score_per_sentence += score
            sentenceValue[sentence[:15]] = total_score_per_sentence / count_words_in_sentence  
            sentences[sentence[:15]] = sentence
            total_sentences_in_all_documents += 1
        
    return total_sentences_in_all_documents, sentences, sentenceValue


def _find_average_score(sentenceValue) -> int:

    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average


def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary += "\n \n" + sentences[sentence].replace('\n', ' ').replace('\r', '')
            sentence_count += 1

    return sentence_count, summary


def run_summarization(documents):

    # 1 Length of documents
    total_documents = len(documents)
    #print(sentences)

    # 2 Create the Frequency matrix of the words in each sentence.
    freq_matrix = _create_frequency_matrix(documents)
    #print(freq_matrix)

    # 3 Calculate TermFrequency and generate a matrix
    tf_matrix = _create_tf_matrix(freq_matrix)
    #print(tf_matrix)

    # 4 creating table for documents per words
    count_doc_per_words = _create_documents_per_words(freq_matrix)
    #print(count_doc_per_words)

    # 5 Calculate IDF and generate a matrix
    idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    #print(idf_matrix)

    # 6 Calculate TF-IDF and generate a matrix
    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    #print(tf_idf_matrix)

    # 7 Important Algorithm: score the sentences
    total_sentences_in_all_documents, sentences, sentence_scores = _score_sentences(tf_idf_matrix, documents)
    #print(len(sentence_scores))

    # 8 Find the threshold
    threshold = _find_average_score(sentence_scores)
    #print(threshold)

    # 9 Important Algorithm: Generate the summary
    sentence_count_result, summary = _generate_summary(sentences, sentence_scores, 1 * threshold)
    return total_sentences_in_all_documents, sentence_count_result, summary


if __name__ == '__main__':
    total_sentences_in_all_documents, sentence_count_result, result = run_summarization(documents)
    print("total number of sentences : ", format(total_sentences_in_all_documents), "\n" )
    print("total number of sentences after Summarization : ", format(sentence_count_result), "\n")
    print(result)

********************* INPUT **********************
URL: https://www.cnn.com/studentnews/article/sitemap-2020-4.html
Date: 2020-04-25
No. of Articles: 2
Keywords: coronavirus
********************* OUTPUT **********************
1
In recent weeks, the Asian city-state has had a dramatic spike in coronavirus infections, with thousands of new cases linked to clusters in foreign worker dormitories. To control the spread, the government has attempted to isolate the dormitories, test workers and move symptomatic patients into quarantine facilities. But those measures have left hundreds of thousands of workers trapped in their dormitories, living cheek by jowl in cramped conditions that make social distancing near impossible.Singapore is home to about 1.4 million migrant workers who come largely from South and Southeast Asia. As housekeepers, domestic helpers, construction workers and manual laborers, these migrants are essential to keeping Singapore functioning -- but are also some of the lowe

In fact, it's those very questions that he is helping us answer.  Sean is helping all of us figure out if it is safe, by putting up his hand first and volunteering. With that injection, Sean had become a critical part of the fastest moving vaccine trials in the history of the world, a vaccine for Covid-19.During a pandemic, the urgency is understandable. In just a few months, the virus has spread to nearly every corner of the globe and sadly taken more lives than several wars or natural disasters combined. It is also true that no one on the planet is immune to this; such is the nature of a novel or new coronavirus. As you process all that, remember that last Thanksgiving, this wasn't even a real concern for human beings, and not even a topic of idle conversation. And, now it is the only thing being discussed in hospitals, boardrooms and at kitchen tables every night, often by Zoom. So, yes. The urgent pace is quite understandable but we have to make sure we can sprint, while also not t